Fetching data from https://www.opendota.com/ about current pro dota scene.

Using free api access (no api-key needed) with 50000 calls per month and 60 calls per minute available. 

More info at: https://www.opendota.com/api-keys.

### Import

In [1]:
import os
import time
import requests
import pandas as pd
import numpy as np

### API Endpoint

In [2]:
BASE_PATH = "https://api.opendota.com/api"
ENDPOINT_GET_PRO_PLAYER_LIST = BASE_PATH + "/proPlayers"
ENDPOINT_GET_PLAYER_INFO = BASE_PATH + "/players/{}"
ENDPOINT_GET_PRO_TEAM_LIST = BASE_PATH + "/teams"
ENDPOINT_GET_PRO_TEAM_PLAYER_LIST = BASE_PATH + "/teams/{}/players"
ENDPOINT_GET_PLAYER_MMR_DISTRIBUTION = BASE_PATH + "/distributions"

### Constant

In [3]:
DATA_DIR = "../data"
STATUS_SUCCESS = 200

### Fetching Raw Data

#### Getting professional player list

In [4]:
PRO_PLAYERS_FILEPATH = os.path.join(DATA_DIR, "pro_players.csv")

if os.path.exists(PRO_PLAYERS_FILEPATH):
    df_players = pd.read_csv(PRO_PLAYERS_FILEPATH)
else:
    pro_players_response = requests.get(ENDPOINT_GET_PRO_PLAYER_LIST)
    df_players = pd.DataFrame(pro_players_response.json())
    df_players.to_csv(PRO_PLAYERS_FILEPATH, index=False)

In [5]:
df_players.head()

,account_id,avatar,avatarfull,avatarmedium,cheese,country_code,fantasy_role,fh_unavailable,full_history_time,is_locked,...,loccountrycode,locked_until,name,personaname,plus,profileurl,steamid,team_id,team_name,team_tag
0,88470,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,0,cn,1,False,2018-11-08T01:14:19.343Z,False,...,CN,NaN,TZY,dy5340978.Tzy丶,True,https://steamcommunity.com/id/misterdurst69/,76561197960354198,6020739,NaN,NaN
1,1296625,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,0,NaN,2,False,2018-06-06T15:08:37.244Z,True,...,US,NaN,Newsham,Newsham,True,https://steamcommunity.com/id/jnewsham/,76561197961562353,6904594,TEAM TEAM,tt
2,3916428,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,0,se,2,True,2018-10-17T14:37:40.444Z,True,...,NaN,NaN,Jerry,theSniffer,NaN,https://steamcommunity.com/profiles/7656119796...,76561197964182156,7237270,Bald,NaN
3,3940262,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,0,NaN,2,False,2018-05-03T08:14:33.014Z,True,...,NaN,NaN,March,March,True,https://steamcommunity.com/id/March8/,76561197964205990,7225724,Team Jinesbrus,TJMM
4,4281729,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,0,NaN,2,False,2019-03-30T04:47:36.136Z,True,...,DK,NaN,syndereN,syndereN,True,https://steamcommunity.com/profiles/7656119796...,76561197964547457,7247587,Anti-MagE-,A-M-


#### Getting professional team list

In [6]:
TEAMS_FILEPATH = os.path.join(DATA_DIR, "teams.csv")

if os.path.exists(TEAMS_FILEPATH):
    df_teams = pd.read_csv(TEAMS_FILEPATH)
else:
    pro_team_response = requests.get(ENDPOINT_GET_PRO_TEAM_LIST)
    df_teams = pd.DataFrame(pro_team_response.json())
    df_teams.to_csv(TEAMS_FILEPATH, index=False)

In [7]:
df_teams.head()

,last_match_time,logo_url,losses,name,rating,tag,team_id,wins
0,1561547432,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,341,Team Secret,1539.59,Secret,1838315,677
1,1561916482,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,641,\tVici Gaming,1516.31,VG,726228,961
2,1561916482,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,488,Team Liquid,1486.50,Liquid,2163,798
3,1561804526,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,702,PSG.LGD,1467.18,PSG.LGD,15,1056
4,1535061995,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,36,VGJ Storm,1450.04,VGJ.Storm,5228654,79


#### Getting active professional player list based on team list

In [8]:
TEAM_PLAYERS_FILEPATH = os.path.join(DATA_DIR, "team_players.csv")

df_team_players = None
if os.path.exists(TEAM_PLAYERS_FILEPATH):
    df_team_players = pd.read_csv(TEAM_PLAYERS_FILEPATH)
else:
    start_time = time.time()
    
    call_count = 0 
    while not os.path.exists(TEAM_PLAYERS_FILEPATH):
        team_id = df_teams["team_id"].iloc[call_count]
        
        try:
            print("\r", "Downloading data of team {}/{}.".format(call_count + 1, df_teams.shape[0]), end="")
            team_players = requests.get(ENDPOINT_GET_PRO_TEAM_PLAYER_LIST.format(team_id))
            
            if team_players.status_code != STATUS_SUCCESS:
                raise Exception("Status code: {}".format(team_players.status_code))
                
            temp = pd.DataFrame(team_players.json())
            temp = temp.loc[temp["is_current_team_member"] == True]
            temp["team_id"] = team_id

            if df_team_players is None:
                df_team_players = temp
            else:
                df_team_players = pd.concat([df_team_players, temp], axis=0)
                
            call_count = call_count + 1
        except:
            min_wait_time = 10
            wait_time = min_wait_time + int(np.ceil(time.time() - start_time))
            
            print("\n\t- Quota reached! Waiting for {} seconds.".format(wait_time))
            time.sleep(wait_time)

            start_time = time.time()
        finally:
            if call_count == len(df_teams["team_id"]):
                df_team_players.reset_index(drop=True, inplace=True)
                df_team_players.to_csv(TEAM_PLAYERS_FILEPATH, index=False)

	- Quota reached! Waiting for 66 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 33 seconds.
	- Quota reached! Waiting for 34 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 28 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 44 seconds.
	- Quota reached! Waiting for 30 seconds.
	- Quota reached! Waiting for 27 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 28 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 27 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 26 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 29 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 29 seconds.
	- Quota reached! Waiting for 11 s

#### Getting pro player mmr ranking estimates

In [9]:
PRO_PLAYERS_MMR_ESTIMATES_FILEPATH = os.path.join(DATA_DIR, "pro_players_mmr_estimates.csv")

df_pro_players_mmr = None
if os.path.exists(PRO_PLAYERS_MMR_ESTIMATES_FILEPATH):
    df_pro_players_mmr = pd.read_csv(PRO_PLAYERS_MMR_ESTIMATES_FILEPATH)
else:
    start_time = time.time()
    
    call_count = 0 
    player_mmr_dict = {"account_id": [], "mmr_estimate": []}
    while not os.path.exists(PRO_PLAYERS_MMR_ESTIMATES_FILEPATH):
        account_id = df_players["account_id"].iloc[call_count]
        
        try:
            print("\r", "Downloading data of player {}/{}.".format(
                call_count + 1, df_players.shape[0]), end="")

            player_info = requests.get(ENDPOINT_GET_PLAYER_INFO.format(account_id))
            
            if player_info.status_code != STATUS_SUCCESS:
                raise Exception("Status code: {}".format(player_info.status_code))
            
            player_info_json = player_info.json()
            player_mmr_dict["account_id"].append(account_id)
            
            if "mmr_estimate" in player_info_json and "estimate" in player_info_json["mmr_estimate"]:
                player_mmr_dict["mmr_estimate"].append(int(player_info_json["mmr_estimate"]["estimate"]))
            else:
                player_mmr_dict["mmr_estimate"].append(np.nan)

            call_count = call_count + 1
        except:
            min_wait_time = 10
            wait_time = min_wait_time + int(np.ceil(time.time() - start_time))
            
            print("\n\t- Quota reached! Waiting for {} seconds.".format(wait_time))
            time.sleep(wait_time)

            start_time = time.time()
        finally:
            if call_count == len(df_players["account_id"]):
                df_pro_players_mmr = pd.DataFrame(player_mmr_dict)
                df_pro_players_mmr.to_csv(PRO_PLAYERS_MMR_ESTIMATES_FILEPATH, index=False)

	- Quota reached! Waiting for 23 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 25 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 28 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 28 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 26 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 25 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 26 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 seconds.
	- Quota reached! Waiting for 11 s

In [10]:
df_pro_players_mmr.head()

,account_id,mmr_estimate
0,88470,6234
1,1296625,6103
2,3916428,7357
3,3940262,7434
4,4281729,6355


#### Getting player mmr distribution

In [11]:
AVG_MMR_RANKS_FILEPATH = os.path.join(DATA_DIR, "mmr_ranks.csv")
AVG_MMR_DISTRIBUTION_FILEPATH = os.path.join(DATA_DIR, "mmr.csv")
AVG_MMR_DISTRIBUTION_BY_COUNTRY_FILEPATH = os.path.join(DATA_DIR, "mmr_country.csv")

if all(os.path.exists(s) for s in [AVG_MMR_RANKS_FILEPATH, 
                                   AVG_MMR_DISTRIBUTION_FILEPATH, 
                                   AVG_MMR_DISTRIBUTION_BY_COUNTRY_FILEPATH]):
    
    df_mmr = pd.read_csv(AVG_MMR_DISTRIBUTION_FILEPATH)
    df_mmr_country = pd.read_csv(AVG_MMR_DISTRIBUTION_BY_COUNTRY_FILEPATH)
    df_mmr_ranks = pd.read_csv(AVG_MMR_RANKS_FILEPATH)
else:
    mmr_distributions = requests.get(ENDPOINT_GET_PLAYER_MMR_DISTRIBUTION)
    df_mmr = pd.DataFrame(mmr_distributions.json()["mmr"]["rows"])
    df_mmr.to_csv(AVG_MMR_DISTRIBUTION_FILEPATH, index=False)
    
    df_mmr_country = pd.DataFrame(mmr_distributions.json()["country_mmr"]["rows"])
    df_mmr_country.to_csv(AVG_MMR_DISTRIBUTION_BY_COUNTRY_FILEPATH, index=False)
    
    df_mmr_ranks = pd.DataFrame(mmr_distributions.json()["ranks"]["rows"])
    df_mmr_ranks.to_csv(AVG_MMR_RANKS_FILEPATH, index=False)

In [12]:
df_mmr.head()

,bin,bin_name,count,cumulative_sum
0,0,0,7491,7491
1,1,100,5392,12883
2,2,200,6492,19375
3,3,300,8163,27538
4,4,400,9484,37022


In [13]:
df_mmr_country.head()

,avg,common,count,loccountrycode
0,4085,AN,1,AN
1,3554,Singapore,12651,SG
2,3399,Vietnam,14988,VN
3,3389,Denmark,3788,DK
4,3376,Seychelles,140,SC


In [14]:
df_mmr_ranks.head()

,bin,bin_name,count,cumulative_sum
0,11,11,36,36
1,12,12,251,287
2,13,13,973,1260
3,14,14,3154,4414
4,15,15,7388,11802


#### Getting TI prizepool values

Data scrapped manually from http://dota2.prizetrac.kr/, and https://en.wikipedia.org/wiki/The_International_(Dota_2).

In [15]:
TI_PRIZEPOOL_FILEPATH = os.path.join(DATA_DIR, "ti_prizeool.csv")

prize_dict = {
    "year": [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
    "prize_dolar": [1600000, 1600000, 2874380, 10931105, 18429613, 20770460, 24787916, 25532177, 28200021]
}

df_ti_prizepool = pd.DataFrame(prize_dict)
df_ti_prizepool.to_csv(TI_PRIZEPOOL_FILEPATH, index=False)

In [16]:
df_ti_prizepool.head()

,year,prize_dolar
0,2011,1600000
1,2012,1600000
2,2013,2874380
3,2014,10931105
4,2015,18429613


#### Players in-game

Source: https://steamcharts.com/app/570#All

Manually scrapping data. 

In [17]:
data_string = """
Month	Avg. Players	Gain	% Gain	Peak Players
June 2019	507,528.4	-40,994.7	-7.47%	865,374
May 2019	548,523.2	+28,304.2	+5.44%	997,341
April 2019	520,219.0	-66,286.9	-11.30%	971,545
March 2019	586,505.8	+21,596.2	+3.82%	1,033,925
February 2019	564,909.7	+89,162.7	+18.74%	964,921
January 2019	475,747.0	+36,379.2	+8.28%	874,888
December 2018	439,367.8	-21,705.7	-4.71%	765,422
November 2018	461,073.5	+29,899.6	+6.93%	826,053
October 2018	431,173.9	-35,296.8	-7.57%	739,643
September 2018	466,470.7	-9,630.3	-2.02%	826,166
August 2018	476,101.1	+34,386.7	+7.78%	829,281
July 2018	441,714.3	-32,185.7	-6.79%	701,582
June 2018	473,900.0	-425.9	-0.09%	796,886
May 2018	474,325.9	+43,984.9	+10.22%	844,713
April 2018	430,340.9	-6,921.4	-1.58%	733,214
March 2018	437,262.3	-1,585.4	-0.36%	773,897
February 2018	438,847.7	-48,014.2	-9.86%	779,299
January 2018	486,861.9	-26,212.4	-5.11%	778,627
December 2017	513,074.3	+25,693.1	+5.27%	864,939
November 2017	487,381.2	+21,254.5	+4.56%	861,173
October 2017	466,126.8	-25,323.4	-5.15%	832,550
September 2017	491,450.2	-65,046.1	-11.69%	829,555
August 2017	556,496.3	+58,051.9	+11.65%	876,395
July 2017	498,444.4	-56,844.7	-10.24%	824,297
June 2017	555,289.1	-12,237.4	-2.16%	923,122
May 2017	567,526.4	+27,248.4	+5.04%	972,876
April 2017	540,278.0	-8,157.4	-1.49%	921,318
March 2017	548,435.4	-43,131.8	-7.29%	956,232
February 2017	591,567.3	+11,285.8	+1.94%	1,040,877
January 2017	580,281.5	-13,639.1	-2.30%	1,007,451
December 2016	593,920.6	+9,669.3	+1.65%	1,014,671
November 2016	584,251.3	-55,103.8	-8.62%	1,007,270
October 2016	639,355.1	+16,771.2	+2.69%	1,141,191
September 2016	622,583.9	-43,429.2	-6.52%	1,064,377
August 2016	666,013.1	+27,800.4	+4.36%	1,117,519
July 2016	638,212.7	-2,014.3	-0.31%	1,084,198
June 2016	640,227.0	+16,428.3	+2.63%	1,095,994
May 2016	623,798.7	-33,145.7	-5.05%	1,075,307
April 2016	656,944.4	-15,610.5	-2.32%	1,164,041
March 2016	672,554.9	-36,623.4	-5.16%	1,291,328
February 2016	709,178.3	+97,003.5	+15.85%	1,248,394
January 2016	612,174.8	+38,830.5	+6.77%	1,067,949
December 2015	573,344.3	+33,807.9	+6.27%	999,452
November 2015	539,536.3	+17,594.6	+3.37%	943,635
October 2015	521,941.7	+13,784.9	+2.71%	917,306
September 2015	508,156.8	-98,787.1	-16.28%	888,728
August 2015	606,944.0	+51,953.0	+9.36%	933,942
July 2015	554,991.0	-13,457.3	-2.37%	877,264
June 2015	568,448.3	-11,900.1	-2.05%	913,997
May 2015	580,348.4	+54,286.7	+10.32%	967,674
April 2015	526,061.7	-45,651.4	-7.99%	929,677
March 2015	571,713.2	-57,257.3	-9.10%	1,213,940
February 2015	628,970.4	+70,466.1	+12.62%	1,262,612
January 2015	558,504.3	+34,564.0	+6.60%	961,737
December 2014	523,940.3	-4,849.5	-0.92%	936,583
November 2014	528,789.8	+33,096.8	+6.68%	963,810
October 2014	495,693.0	+17,694.6	+3.70%	880,655
September 2014	477,998.5	-12,885.4	-2.62%	864,261
August 2014	490,883.9	-46,134.8	-8.59%	774,319
July 2014	537,018.7	+23,235.6	+4.52%	874,975
June 2014	513,783.1	+31,395.8	+6.51%	833,145
May 2014	482,387.2	+60,677.0	+14.39%	843,024
April 2014	421,710.2	+11,954.7	+2.92%	734,998
March 2014	409,755.6	-11,358.6	-2.70%	698,197
February 2014	421,114.2	+27,253.9	+6.92%	738,682
January 2014	393,860.3	+27,253.8	+7.43%	673,496
December 2013	366,606.5	+18,360.1	+5.27%	685,503
November 2013	348,246.4	+18,568.7	+5.63%	702,792
October 2013	329,677.6	+17,252.9	+5.52%	581,615
September 2013	312,424.8	-18,295.3	-5.53%	566,715
August 2013	330,720.1	+92,920.0	+39.07%	520,532
July 2013	237,800.1	+27,575.3	+13.12%	422,617
June 2013	210,224.8	+15,861.0	+8.16%	326,160
May 2013	194,363.8	+19,528.1	+11.17%	325,815
April 2013	174,835.7	-6,043.2	-3.34%	299,667
March 2013	180,878.9	+13,905.9	+8.33%	325,598
February 2013	166,973.0	+19,224.8	+13.01%	283,870
January 2013	147,748.1	+25,823.7	+21.18%	260,989
December 2012	121,924.4	+20,847.0	+20.62%	213,521
November 2012	101,077.4	+25,112.0	+33.06%	169,631
October 2012	75,965.4	+14,097.8	+22.79%	171,860
September 2012	61,867.7	+6,099.1	+10.94%	118,724
August 2012	55,768.6	+3,047.6	+5.78%	108,689
July 2012	52,721.1	-	-	75,041
"""

In [18]:
ACTIVITY_FILEPATH = os.path.join(DATA_DIR, "game_activity.csv")

months = ["January", "February", "March", "April", "May", "June", "July", "August", 
          "September", "October", "November", "December"]

activity_dict = {"month": [], "year": [], "average_activity": [], "peak_activity": []}
for i, row in enumerate(data_string.split("\n")):
    row_parts = row.split()
    
    if any((m in row_parts) for m in months):
        activity_dict["month"].append(row_parts[0])
        activity_dict["year"].append(int(row_parts[1]))
        activity_dict["average_activity"].append(float(row_parts[2].replace(",", "")))
        activity_dict["peak_activity"].append(float(row_parts[5].replace(",", "")))
        
df_activity = pd.DataFrame(activity_dict)
df_activity.to_csv(ACTIVITY_FILEPATH, index=False)

In [19]:
df_activity.head()

,month,year,average_activity,peak_activity
0,June,2019,507528.4,865374.0
1,May,2019,548523.2,997341.0
2,April,2019,520219.0,971545.0
3,March,2019,586505.8,1033925.0
4,February,2019,564909.7,964921.0


### Cleaning Data

- preparing `df_teams` and `df_players` dataframes to join

In [20]:
df_teams.rename(columns={"last_match_time": "team_last_match_time"}, inplace=True)
df_teams.drop(columns=["name"], inplace=True)
df_teams = df_teams.set_index("team_id")
df_players = df_players.set_index("team_id")

- creating `df_total_players` dataframe which will store all players data

In [21]:
df_total_players = df_players.join(df_teams, on="team_id")

- preparing `df_total_players` and `df_team_players` to join

In [22]:
df_total_players.reset_index(inplace=True)
df_total_players.set_index("account_id")

df_team_players.rename(columns={
    "wins": "team_wins",
    "games_played": "team_games_played"
}, inplace=True)

df_team_players = df_team_players[["account_id", "team_games_played", "team_wins", "is_current_team_member"]]
df_team_players = df_team_players.set_index("account_id")

- joining `df_total_players` with `df_team_players`

In [23]:
df_total_players = df_total_players.join(df_team_players, on="account_id")

- preparing `df_pro_players_mmr` and `df_total_players` to join

In [24]:
df_pro_players_mmr = df_pro_players_mmr.set_index("account_id")

- join `df_pro_players_mmc` and `df_total_players`

In [25]:
df_total_players = df_total_players.join(df_pro_players_mmr, on="account_id")

- leave only subset of columns

In [26]:
df_total_players.columns

Index(['team_id', 'account_id', 'avatar', 'avatarfull', 'avatarmedium',
       'cheese', 'country_code', 'fantasy_role', 'fh_unavailable',
       'full_history_time', 'is_locked', 'is_pro', 'last_login',
       'last_match_time', 'loccountrycode', 'locked_until', 'name',
       'personaname', 'plus', 'profileurl', 'steamid', 'team_name', 'team_tag',
       'team_last_match_time', 'logo_url', 'losses', 'rating', 'tag', 'wins',
       'team_games_played', 'team_wins', 'is_current_team_member',
       'mmr_estimate'],
      dtype='object')

In [27]:
columns_to_keep = [
    "team_id", "account_id", "country_code", "fantasy_role", "full_history_time", "is_pro",
    "last_login", "last_match_time", "loccountrycode", "is_locked", "locked_until", "name", 
    "team_name", "team_last_match_time", "losses", "rating", "wins", "team_games_played", 
    "team_wins", "is_current_team_member", "mmr_estimate"
]

df_total_players = df_total_players[columns_to_keep]

- saving cleaned data

In [28]:
CLEANED_PLAYERS_FILEPATH = os.path.join(DATA_DIR, "players.csv")
df_total_players.to_csv(CLEANED_PLAYERS_FILEPATH, index=False)

- removing temporary files

In [29]:
!rm $PRO_PLAYERS_FILEPATH
!rm $TEAMS_FILEPATH
!rm $TEAM_PLAYERS_FILEPATH
!rm $PRO_PLAYERS_MMR_ESTIMATES_FILEPATH

- data dir files

In [30]:
!ls $DATA_DIR

game_activity.csv mmr_country.csv   players.csv
mmr.csv           mmr_ranks.csv     ti_prizeool.csv
